In [1]:
import pandas as pd

from statsmodels.formula.api import ols
from scipy.stats import ttest_ind
from statsmodels.iolib.summary2 import summary_col

In [7]:
def regression(df, years, model, typ=None):
    if typ is not None:
        df = df.query(f'{typ} == 1')
    
    toTest = [df[df.EditorYear == year] for year in years]
    toTest = pd.concat(toTest, ignore_index=True, sort=False)
    toTest = toTest.assign(isAfter = toTest.EditorYear > 0)
    print(toTest.shape)
    
    model = ols(model, data=toTest)
    
    #results = model.fit(cov_type = 'HAC', cov_kwds={'maxlags':1})
    results = model.fit(cov_type = 'cluster', cov_kwds={'groups': toTest['NewAuthorId']})
    
    
    return results

In [23]:
def printRow(res, predictor='', name='', vspace=0.2, bottomRow=False):
    
    if bottomRow:
        print(f'Observations & {int(res.nobs)} \\\\')
        print(f'Adjusted $R^2$ & {round(res.rsquared, 3)} \\\\')
        #out += '\multirow{3}{*}{' + '{:,}'.format(int(res.nobs))+ '}'+ ' & \multirow{3}{*}{' + f'{round(res.rsquared, 3)}' + '}'+ '\\vspace*{'+f'{vspace}'+'cm}\\\\'
    
    else:
        beta = round(res.params[predictor], 3)
        lo = round(res.conf_int().loc[predictor][0], 3)
        hi = round(res.conf_int().loc[predictor][1], 3)
        pval = res.pvalues[predictor]
        pval = f'{round(pval, 3)}' if pval >= 0.001 else '{:e}'.format(pval) #'$< 0.001$'

        out = f'{name} & ' + f'{beta} & {(lo, hi)} & {pval} &'

        out += ' & \\vspace*{'+f'{vspace}'+'cm}\\\\'
        print(out)

In [12]:
toPlot = pd.read_csv('../data/figure_3_selfpub_regression.csv')
toPlot.shape

(119553, 8)

In [13]:
toPlot.head()

,NewAuthorId,issn,EditorYear,JournalCount,Productivity,Percentage,gender,start_year
0,0,0,-4,1.0,2.0,0.5,male,1956
1,0,0,-3,0.0,1.0,0.0,male,1956
2,0,0,-2,0.0,NaN,0.0,male,1956
3,0,0,-1,0.0,NaN,0.0,male,1956
4,0,0,0,0.0,3.0,0.0,male,1956


In [14]:
toPlot = toPlot.assign(JournalCount = lambda df: (df.JournalCount-df.JournalCount.mean())/df.JournalCount.std())

toPlot.shape # 119553

(119553, 8)

In [15]:
%%time
resCount = regression(
    toPlot,
    [-1,-2,-3,-4,0,1,2,3,4,5],
    'JournalCount ~ start_year + EditorYear + isAfter + EditorYear:isAfter + gender + isAfter:gender + C(issn) '
)

(119553, 9)
CPU times: user 45.7 s, sys: 5.21 s, total: 50.9 s
Wall time: 30.8 s


In [16]:
summary = summary_col(
    [resCount],
    stars=True,
    model_names = ['Regression'],
    float_format='%0.3f',
    info_dict={
        'N':lambda x: "{0:,}".format(int(x.nobs)),
    },
    regressor_order=['EditorYear', 'EditorYear:isAfter[T.True]', 'isAfter[T.True]', 'gender[T.male]',
                    'isAfter[T.True]:gender[T.male]'
                    ],
    drop_omitted=True
)
summary

,Regression
EditorYear,0.029***
,(0.002)
EditorYear:isAfter[T.True],-0.031***
,(0.004)
isAfter[T.True],0.053***
,(0.016)
gender[T.male],0.046***
,(0.013)
isAfter[T.True]:gender[T.male],0.035**
,(0.016)


In [24]:
printRow(resCount, 'isAfter[T.True]', 'After editorship starts ($\\beta_1$)')
printRow(resCount, 'EditorYear', 'Year since editorship starts ($\\beta_2$)')
printRow(resCount, 'EditorYear:isAfter[T.True]', 'After editorship starts $\\times$ Year since editorship starts ($\\beta_3$)')
printRow(resCount, 'gender[T.male]', 'Male ($\\beta_4$)')    
printRow(resCount, 'isAfter[T.True]:gender[T.male]', 'After editorship starts $\\times$ Male ($\\beta_5$)', 0.4)    
printRow(resCount, bottomRow=True)

After editorship starts ($\beta_1$) & 0.053 & (0.021, 0.086) & 0.001 & & \vspace*{0.2cm}\\
Year since editorship starts ($\beta_2$) & 0.029 & (0.025, 0.034) & 2.745745e-43 & & \vspace*{0.2cm}\\
After editorship starts $\times$ Year since editorship starts ($\beta_3$) & -0.031 & (-0.038, -0.024) & 2.921876e-18 & & \vspace*{0.2cm}\\
Male ($\beta_4$) & 0.046 & (0.02, 0.071) & 5.051811e-04 & & \vspace*{0.2cm}\\
After editorship starts $\times$ Male ($\beta_5$) & 0.035 & (0.004, 0.067) & 0.027 & & \vspace*{0.4cm}\\
Observations & 119553 \\
Adjusted $R^2$ & 0.167 \\


In [20]:
%%time
resRate = regression(
    toPlot,
    [-1,-2,-3,-4,0,1,2,3,4,5],
    'Percentage ~ start_year + EditorYear + isAfter + EditorYear:isAfter + gender + isAfter:gender + C(issn) '
)

(119553, 9)
CPU times: user 46.3 s, sys: 4.8 s, total: 51.1 s
Wall time: 31 s


In [21]:
summary = summary_col(
    [resRate],
    stars=True,
    model_names = ['Regression'],
    float_format='%0.3f',
    info_dict={
        'N':lambda x: "{0:,}".format(int(x.nobs)),
    },
    regressor_order=['EditorYear', 'EditorYear:isAfter[T.True]', 'isAfter[T.True]', 'gender[T.male]',
                    'isAfter[T.True]:gender[T.male]'
                    ],
    drop_omitted=True
)
summary

,Regression
EditorYear,0.003***
,(0.000)
EditorYear:isAfter[T.True],-0.004***
,(0.001)
isAfter[T.True],0.008***
,(0.003)
gender[T.male],-0.005**
,(0.002)
isAfter[T.True]:gender[T.male],0.007**
,(0.003)


In [25]:
printRow(resRate, 'isAfter[T.True]', 'After editorship starts ($\\beta_1$)')
printRow(resRate, 'EditorYear', 'Year since editorship starts ($\\beta_2$)')
printRow(resRate, 'EditorYear:isAfter[T.True]', 'After editorship starts $\\times$ Year since editorship starts ($\\beta_3$)')
printRow(resRate, 'gender[T.male]', 'Male ($\\beta_4$)')    
printRow(resRate, 'isAfter[T.True]:gender[T.male]', 'After editorship starts $\\times$ Male ($\\beta_5$)', 0.4)    
printRow(resRate, bottomRow=True)

After editorship starts ($\beta_1$) & 0.008 & (0.002, 0.014) & 0.008 & & \vspace*{0.2cm}\\
Year since editorship starts ($\beta_2$) & 0.003 & (0.002, 0.003) & 6.276878e-11 & & \vspace*{0.2cm}\\
After editorship starts $\times$ Year since editorship starts ($\beta_3$) & -0.004 & (-0.006, -0.003) & 5.165952e-12 & & \vspace*{0.2cm}\\
Male ($\beta_4$) & -0.005 & (-0.01, -0.001) & 0.025 & & \vspace*{0.2cm}\\
After editorship starts $\times$ Male ($\beta_5$) & 0.007 & (0.001, 0.012) & 0.017 & & \vspace*{0.4cm}\\
Observations & 119553 \\
Adjusted $R^2$ & 0.087 \\
